In [9]:
import pandas as pd

df = pd.read_excel('MCMB数据[solve-regionB].xlsx')
df

,年,人口密度（人/平方千米）,耕地面积,森林面积,大象致人伤亡数量,食草动物致人伤亡数量,食肉动物致人伤亡数量,营地累计数量和住宿（MMNR内部以及周边地区）,犀牛栖息地百分比,牛放牧扩大占比
0,1971,15.82,9193.94,89274.32,0,0,0,1,35.0,9.0
1,1972,16.10,9898.26,89100.47,0,0,0,1,35.7,9.1
2,1973,19.50,10911.67,84540.76,0,0,0,1,35.3,9.1
3,1974,19.68,10801.35,83303.77,0,0,0,1,34.0,9.3
4,1975,20.41,12312.00,85281.00,0,0,0,3,33.6,9.6
5,1976,23.39,11472.47,79465.74,0,0,0,3,33.0,9.5
6,1977,26.63,11838.97,75772.09,0,0,0,3,32.0,9.8
7,1978,28.45,12635.86,72695.70,0,0,0,3,31.7,9.7
8,1979,26.00,15721.21,72952.16,0,0,0,3,31.2,10.0
9,1980,30.46,14593.85,71764.92,0,0,0,10,31.0,10.8


In [48]:
from scipy.optimize import minimize
import numpy as np
import math


def getFun(args):
    k1, k2, k3 = args
    fun1 = lambda x: k3 * math.exp(x[2])
    fun2 = lambda x: k1 * math.exp(x[0]) + k2 * math.exp(x[1])

    f = lambda x: -(fun1(x) / (fun1(x) + fun2(x)) - fun2(x) / (fun1(x) + fun2(x)))
    return f


def getCon(args):
    argsKey = list(args.keys())
    x1max = args['第2000年营地'] # A
    x1min = x1max - args['at']

    x2max = args['第2000年牛放牧扩大占比'] # B
    x2min = x2max - args['bt']

    x3min = 0 # C
    x3max = 17
    cons = (
        {'type': 'ineq', 'fun': lambda x: x[0] - x1min},
        {'type': 'ineq', 'fun': lambda x: -x[0] + x1max},

        {'type': 'ineq', 'fun': lambda x: x[1] - x2min},
        {'type': 'ineq', 'fun': lambda x: -x[1] + x2max},

        {'type': 'ineq', 'fun': lambda x: x[2] - x3min},
        {'type': 'ineq', 'fun': lambda x: -x[2] + x3max},

        {'type': 'ineq', 'fun': lambda x: (args['第2000年犀牛栖息地百分比'] + x[2]) / (args['第2000年牛放牧扩大占比'] - x[1]) -
                                          args['sum1948-2000(ci/bi)'] / (2000 - 1948 + 1)},
    )
    return cons


# k1, k2, k3
argsK = (1, 1, 1)

search2000 = df.query('年 == 2000')
year2000 = search2000.loc[search2000.index.values[0]].to_dict()
sum19482000cibi = 0
for i, row in df.iterrows():
    sum19482000cibi += row['犀牛栖息地百分比'] / row['牛放牧扩大占比']

argsCon = {
    '第2000年营地': year2000['营地累计数量和住宿（MMNR内部以及周边地区）'], # A
    '第2000年牛放牧扩大占比': year2000['牛放牧扩大占比'], # B
    '第2000年犀牛栖息地百分比': year2000['犀牛栖息地百分比'], # C
    'at':10,
    'bt':21,
    'sum1948-2000(ci/bi)': sum19482000cibi
}
print(argsCon)

# 变量初始值
x0 = np.asarray((1, 1, 1))
res = minimize(getFun(argsK), x0, constraints=getCon(argsCon))

print(res)

{'第2000年营地': 23.0, '第2000年牛放牧扩大占比': 17.0, '第2000年犀牛栖息地百分比': 18.0, 'at': 10, 'bt': 21, 'sum1948-2000(ci/bi)': 76.02553497600856}
     fun: 0.999908396753208
     jac: array([ 9.15899873e-05,  0.00000000e+00, -9.16048884e-05])
 message: 'Optimization terminated successfully'
    nfev: 8
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([13.        ,  2.63555249,  3.00888631])


# 第一问区域B求解结果
x1 = 13.00
x2 = 2.64
x3 = 3.00

因此针对于2000年：

对于B区域，应该采取如下方案：
1. 减少13.00%的营地
2. 减少2.64%的放牧
3. 增加3.00%的犀牛栖息地